## Import the library

In [ ]:
#!pip install datasets

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW



In [ ]:
data = pd.read_csv("train.csv")
data.head()
del data['Unnamed: 0']

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=100, shuffle=True)

train_data.to_csv('train_clean.csv',index=False)
test_data.to_csv('test_clean.csv',index=False)

## Setting the model, tokenizer and preprocessing the dataset

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'train_clean.csv','test':'test_clean.csv',})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset.rename_column('output', 'labels')

In [ ]:
# function to tokenize the data
def preprocess_function(records):
    return tokenizer(records['context'], records['target'], truncation=True, max_length = 128)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True, )

Map:   0%|          | 0/10816 [00:00<?, ? examples/s]

Map:   0%|          | 0/2705 [00:00<?, ? examples/s]

In [ ]:
pd.DataFrame(encoded_dataset['train']).head()

,context,target,labels,input_ids,token_type_ids,attention_mask
0,그러나 이들은 국가 예산 반영과 증액을 부탁하는 전북 정치권에 분에 넘치는 친절함을...,전북에는 친절한 사람들이 많다.,1,"[4479, 12465, 1132, 4749, 11689, 440, 1263, 28...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,그는 구체적인 내용에 대해서는 공수처장 추천위원의 인원수를 늘리는 방향으로 하는 것...,민주당은 공수처장 후보추천위원회의 인원수를 바꿀 예정이다.,1,"[16514, 28320, 2861, 266, 9125, 28630, 5348, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,중마동 순환버스 번 번은 길호상가 내를 경유하도록 조정하고 번 노선은 옥곡면 장동마...,중마동 순환버스 번 번은 길호상가 내를 번 노선은 옥곡면 장동마을을 -번 노선은 다...,1,"[513, 528, 476, 1694, 1318, 14354, 2488, 2488,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,정기적 전기안전 점검활동을 하는 전기안전연구원(한국전기안전공사)과 협력해 리콜 대상...,전기안전연구원은 위니아딤채에게 제품안전기본법에 따른 권고조치를 하였다.,1,"[333, 27469, 4113, 20425, 21136, 28124, 644, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,일 시에 따르면 개 읍면동 주민자치센터와 전체 개 경로당 시립도서관 개소 공공 문화...,임시 휴관이 언제 끝날지는 확정되지 않았다.,0,"[537, 472, 266, 19807, 461, 12176, 703, 1665, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['context'])
encoded_dataset = encoded_dataset.remove_columns(['target'])
encoded_dataset.set_format("torch")

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,)

In [ ]:
train_dataloader = DataLoader(
    encoded_dataset['train'], shuffle = True, batch_size = 16, collate_fn = data_collator
)

In [ ]:
train_dataloader

In [ ]:
from transformers import get_scheduler
from transformers import AdamW
# Setting the optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5)

#  Setting the learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


2028


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
# Training
from tqdm.auto import tqdm # for checking the progress

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k : v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/2028 [00:00<?, ?it/s]

## Training and evaluate

In [ ]:
eval_dataloader = DataLoader(
    encoded_dataset['test'], shuffle = True, batch_size = 16, collate_fn = data_collator
)

In [ ]:
# Evaluating
import evaluate

all_predictions = []

m1 = evaluate.load("f1")
m2 = evaluate.load("accuracy")

total_loss = 0
num_batch = 0

model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim= -1)

  loss = outputs.loss
  total_loss += loss.item()
  num_batch += 1

  m1.add_batch(predictions = predictions, references = batch["labels"])
  m2.add_batch(predictions = predictions, references = batch["labels"])
  all_predictions.extend(predictions.cpu().numpy())

f1 = m1.compute()
acc = m2.compute()
average_loss = total_loss / num_batch


In [ ]:
from transformers import trainer
model.save_pretrained('C:\Workspace\anus')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('C:\Workspace\anus')

In [ ]:
print(f"Accuracy: {acc['accuracy']} F1: {f1['f1']} Average Loss: {average_loss}")

Accuracy: 0.6916820702402957 F1: 0.6058601134215501 Average Loss: 0.659484916399507


## Verifying Hypothesis

In [ ]:
dataset1 = load_dataset('csv', data_files={'entail': 'entailment.csv', 'con' : 'contradiction.csv',})

Generating ental split: 0 examples [00:00, ? examples/s]

Generating con split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset1 = dataset1.rename_column('label', 'labels')

In [ ]:
encoded_dataset = dataset1.map(preprocess_function, batched=True, )

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['context'])
encoded_dataset = encoded_dataset.remove_columns(['target'])
encoded_dataset.set_format("torch")

In [ ]:
entail_dataloader = DataLoader(
    encoded_dataset['entail'], shuffle = True, batch_size = 16, collate_fn = data_collator
)
con_dataloader = DataLoader(
    encoded_dataset['con'], shuffle = True, batch_size = 16, collate_fn = data_collator
)

In [ ]:
# Evaluating
import evaluate

all_predictions = []

m1 = evaluate.load("f1")
m2 = evaluate.load("accuracy")

total_loss = 0
num_batch = 0

model.eval()
for batch in entail_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim= -1)

  loss = outputs.loss
  total_loss += loss.item()
  num_batch += 1

  m1.add_batch(predictions = predictions, references = batch["labels"])
  m2.add_batch(predictions = predictions, references = batch["labels"])
  all_predictions.extend(predictions.cpu().numpy())

f1 = m1.compute()
acc = m2.compute()
average_loss = total_loss / num_batch

In [ ]:
all_predictions

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
# Evaluating
import evaluate

all_predictions = []

m1 = evaluate.load("f1")
m2 = evaluate.load("accuracy")

total_loss = 0
num_batch = 0

model.eval()
for batch in con_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim= -1)

  loss = outputs.loss
  total_loss += loss.item()
  num_batch += 1

  m1.add_batch(predictions = predictions, references = batch["labels"])
  m2.add_batch(predictions = predictions, references = batch["labels"])
  all_predictions.extend(predictions.cpu().numpy())

f1 = m1.compute()
acc = m2.compute()
average_loss = total_loss / num_batch

In [ ]:
all_predictions

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1]

In [ ]:
print(f"Accuracy: {acc['accuracy']} F1: {f1['f1']} Average Loss: {average_loss}")

Accuracy: 0.8636363636363636 F1: 0.9268292682926829 Average Loss: 0.2063917020956675
